In [27]:
from cleaning_script import prep_data
from model1 import my_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train,y_train,df_test,y_test= prep_data()

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
model = RandomForestClassifier()
model.fit(df_train,y_train)

print("TRAIN on RF")
print("ROC_AUC score: ", cross_val_score(model,df_train,y_train,cv=5,scoring='roc_auc').mean())
print("Accuracy score: ", cross_val_score(model,df_train,y_train,cv=5).mean())
print("f1 score: ", cross_val_score(model,df_train,y_train,cv=5,scoring='f1').mean())

y_pred = model.predict(df_test)

print("TEST on RF")
print("ROC_AUC score: ", roc_auc_score(y_test,y_pred))
print("Accuracy score: ", accuracy_score(y_test,y_pred))
print("f1 score: ", f1_score(y_test,y_pred))


TRAIN on RF
ROC_AUC score:  0.8895859930397279
Accuracy score:  0.8186505759736697
f1 score:  0.7387805885975097
TEST on RF
ROC_AUC score:  0.7939978692545664
Accuracy score:  0.8175946547884188
f1 score:  0.7369097333761646


In [16]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=9, min_samples_leaf=13, min_samples_split=2)
model.fit(df_train,y_train)

print("TRAIN on DT")
print("ROC_AUC score: ", cross_val_score(model,df_train,y_train,cv=5,scoring='roc_auc').mean())
print("Accuracy score: ", cross_val_score(model,df_train,y_train,cv=5).mean())
print("f1 score: ", cross_val_score(model,df_train,y_train,cv=5,scoring='f1').mean())

y_pred = model.predict(df_test)

print("TEST on DT")
print("ROC_AUC score: ", roc_auc_score(y_test,y_pred))
print("Accuracy score: ", accuracy_score(y_test,y_pred))
print("f1 score: ", f1_score(y_test,y_pred))

TRAIN on DT
ROC_AUC score:  0.8764307588680811
Accuracy score:  0.8088315962698847
f1 score:  0.7396114980886782
TEST on DT
ROC_AUC score:  0.7893913380116717
Accuracy score:  0.8062360801781737
f1 score:  0.730399752091726


In [20]:
param_grid={'n_estimators': [300,400,500],
            'max_depth': [3, None,1,2,5],
            'max_features': ['sqrt','log2', None]
           }

grid = GridSearchCV(RandomForestClassifier(),param_grid=param_grid,n_jobs=-1,scoring="f1",cv=5)
grid.fit(df_train,y_train)

best_estimator = grid.best_params_

In [24]:
best_rf_model = grid.best_estimator_
best_rf_model.fit(df_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
print("TRAIN on RF_GS")
print("ROC_AUC score: ", cross_val_score(best_rf_model,df_train,y_train,cv=5,scoring='roc_auc').mean())
print("Accuracy score: ", cross_val_score(best_rf_model,df_train,y_train,cv=5).mean())
print("f1 score: ", cross_val_score(best_rf_model,df_train,y_train,cv=5,scoring='f1').mean())

y_pred = best_rf_model.predict(df_test)

print("TEST on RF_GS")
print("ROC_AUC score: ", roc_auc_score(y_test,y_pred))
print("Accuracy score: ", accuracy_score(y_test,y_pred))
print("f1 score: ", f1_score(y_test,y_pred))

TRAIN on RF_GS
ROC_AUC score:  0.9122870603162669
Accuracy score:  0.8368074602303895
f1 score:  0.7801648797902647
TEST on RF_GS
ROC_AUC score:  0.8322512737964819
Accuracy score:  0.8406458797327394
f1 score:  0.78373885446577


In [31]:
#gb grid search
gb_grid = {'n_estimators': [10,50,100],
                  'min_samples_leaf': [1, 7, 9, 13],
                  'max_depth': [3, 4, 5, 6, 7],
                  'max_features': [2, 5, 10, 12,15],
                  'learning_rate': [0.05, 0.02, 0.01,.1,1]}
gb_gridsearch = GridSearchCV(GradientBoostingClassifier(),
                             gb_grid,
                             n_jobs=-1,
                             verbose=True,
                             scoring='neg_mean_squared_error')
gb_gridsearch.fit(df_train, y_train)

print( "best parameters:", gb_gridsearch.best_params_ )

best_gb_model = gb_gridsearch.best_estimator_

Fitting 3 folds for each of 1500 candidates, totalling 4500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 28.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 37.3min
[Parallel(n_jobs=-1)]: Done 4500 out of 4500 | elapsed: 42.7min finished


best parameters: {'learning_rate': 0.1, 'max_depth': 6, 'max_features': 5, 'min_samples_leaf': 7, 'n_estimators': 100}


In [32]:
best_gb_model.fit(df_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=6,
              max_features=5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=7, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [33]:
print("TRAIN on GB_GS")
print("ROC_AUC score: ", cross_val_score(best_gb_model,df_train,y_train,cv=5,scoring='roc_auc').mean())
print("Accuracy score: ", cross_val_score(best_gb_model,df_train,y_train,cv=5).mean())
print("f1 score: ", cross_val_score(best_gb_model,df_train,y_train,cv=5,scoring='f1').mean())

y_pred = best_gb_model.predict(df_test)

print("TEST on GB_GS")
print("ROC_AUC score: ", roc_auc_score(y_test,y_pred))
print("Accuracy score: ", accuracy_score(y_test,y_pred))
print("f1 score: ", f1_score(y_test,y_pred))

TRAIN on GB_GS
ROC_AUC score:  0.9102661729729448
Accuracy score:  0.8352715304443226
f1 score:  0.7773797054652114
TEST on GB_GS
ROC_AUC score:  0.8233274854103364
Accuracy score:  0.8332962138084633
f1 score:  0.7728031567764456
